# GEOG5414M Programming for Geographical Information Analysis <a class="tocSkip">

#### Contact: F.L.Pontin@leeds.ac.uk <a class="tocSkip">

# Getting started with spatial data

Up until now all the packages we have needed have already been installed in the default google colab environment. Now we have two further packages that we are going to use: contextily and geoplot, that are not installed in the default environment.

To install packages from within a notebook the quickest way is to use: !pip install PACKAGENAME

- the ! character is used to execute shell commands directly from a code cell. This allows you to run commands that you would normally run in a terminal or command prompt.
- pip is a Python package installer. It's used to install and manage software packages written in Python.
- install: a command for pip that tells it to install a package
- PACKAGENAME e.g. contextily or geoplot: examples of packages we can install

In [ ]:
# #  run these lines of code
!pip install contextily
!pip install geoplot
!pip install git+https://github.com/pmdscully/geo_northarrow.git

As is coding custom we import the required packages at the beginning<br>
<font color='orchid'> <b>Import the packages below </font>

In [ ]:
#Import the required packages

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import contextily as ctx
import seaborn as sns

import geoplot as gplt
import geoplot.crs as gcrs
from geo_northarrow import add_north_arrow


We have imported a new package 'geopandas'. Geopandas works like pandas but also handles spatial data. Geopandas was designed to allow people to easily handle and use spatial datasets in Python. <br>



## Reading in spatial data

Geopandas data frames work the same way as pandas data frames, but are also able to handle the spatial element of the dataset. Therefore, the code we use is very similar. We use <code>geopandas.read_file(<font color =red>file_path</font>) </code> replacing <font color =red>\"file_path\"</font>, with the actual path to the shape data file you want to use, in this case .geojson files).

We are going to read the files from the GitHub repo. The urls to the data on GitHub are:
- https://github.com/FrancescaPontin/GEOG5990M/blob/main/data/week_6_7/eeds_travel_to_work_mode_distance.geojson
-https://github.com/FrancescaPontin/GEOG5990M/blob/main/data/week_6_7/West_Yorkshire_bus_stops.geojson

Remember, when we are reading in data from GitHub we need to amend the link from: '/blob/' to: '/raw/refs/heads/'


<font color='orchid'> Run the code below to read the spatial data in </font>

In [ ]:
bus_stops =gpd.read_file('https://github.com/FrancescaPontin/GEOG5990M/raw/refs/heads/main/data/week_6_7/West_Yorkshire_bus_stops.geojson')
leeds=gpd.read_file('https://github.com/FrancescaPontin/GEOG5990M/raw/refs/heads/main/data/week_6_7/leeds_travel_to_work_mode_distance.geojson')


### Data Exploration
Just as we have done before we are going to explore the data by having a look at the data frames and by visualising both the data sets.

In [ ]:
# let's have a look at the bus_stops geopandas dataframe
bus_stops.head()

### West Yorkshire Bus Stops

https://datamillnorth.org/dataset/vqxk4/west-yorkshire-bus-stops

The data includes:
- 'STOPID': a unique ID for each bus stop
- 'NAME': the name of each bus stop
- 'TOWN': the town in which the bus stop is located
- '0730__0930'...'2000_2100': average number of buses per hour during the AM Peak (0730 - 0930) Inter-Peak (1200-1400), PM Peak (1500 - 1830) and evening (2000 - 2100) time periods.
- 'X' and 'Y': coordinates
- 'Z": elevation of the bus stop.
- PhotoF: link to a front photo of the bus stop
- PhotoS: link to a side photo of the bus stop

- 'geometry' listing the type of geometry, in this case POINT. And two numbers - the coordinates of the point.




In [ ]:
# let's have a look at the Leeds geopandas dataframe
leeds.head()

### Leeds
LSOA level polygon file for Leeds with 2021 census data bout travel to work distance and transport mode- downloaded from [geoportal](https://geoportal.statistics.gov.uk/datasets/ons::lower-layer-super-output-areas-december-2021-boundaries-ew-bfc-v10-2/about): 
- Notice this time the 'geometry' column has more than two numbers and 'POLYGON()' rather than 'POINT()'

## Simple spatial data visualisation
### Point data

In [ ]:
# Plot the bus stop data

# define the plot size and nummber of subplots (1 i.e. 1 plot)
f, ax = plt.subplots(1, figsize=(16, 8))
# plot the  bus stop, specifying the subplot axis
bus_stops.plot(ax=ax)
# show the plot
plt.show()

### Polygon data

In [ ]:
# Plot the leeds data

# define the plot size and nummber of subplots (1 i.e. 1 plot)
f, ax = plt.subplots(1, figsize=(16, 8))
# plot the  leeds data, specifying the subplot axis
leeds.plot(ax=ax)
# show the plot
plt.show()

<b>Note</b> the geometry column this time is made up of POLYGON data, made up of many coordinate points. <br>
Let us have a closer look at the polygon geometry. We are going to look at the geometry of row 1.

In [ ]:
# Lets look at index 3 (for me it is 'Leeds 009B') of the geometry column
leeds.loc[3,:]

In [ ]:
# Now let us look at the geometry column
# Notice the shape is quite complex with lots of edges
leeds.loc[3,'geometry']

In [ ]:
# let us look at the list of coordinates
print(leeds.loc[3,'geometry'])
# Each point is a corner of the LSOA

### geopandas.explore()

<code>gpd.explore()</code> is a really useful function that generate an interactive leaflet map based on GeoDataFrame. It is particularly useful for initial data exploration. The function creates:
- a navigable map
- with 'zoom in and out' functionality
- and hover over function displaying the variables in the GeoDataFrame

The <code>.explore()</code> has many customisable parameters, see the [package documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html)

In [ ]:
bus_stops.explore()

In [ ]:
leeds.explore()

## Coordinate Reference Systems (CRS)
Before we map multiple layers we need to check they have the same Coordinate Reference System (CRS). Using the <code>.crs</code> function.

This displays useful information about the projection used, which areas it can be used in etc.

In [ ]:
leeds.crs

Look at what happens when we change the projection

In [ ]:
# NAD83 Canadian Spatial Reference System: Large and medium scale topographic mapping and engineering survey.
leeds_new_proj= leeds.to_crs(epsg=2953)

# EPSG:3851 New Zealand Geodetic Datum 2000: Spatial referencing and conformal mapping on the NZ continental shelf.
leeds_new_proj2= leeds.to_crs(epsg=3851)

In [ ]:
leeds_new_proj.crs

In [ ]:
# plot the two diifferent new projections alongside the original projection
f, ax = plt.subplots(1,3, figsize=(30, 20))
leeds.plot(ax=ax[0])
leeds_new_proj.plot(ax=ax[1])
leeds_new_proj2.plot(ax=ax[2])
plt.show()
# Note the different scales on the axis and orientation of the LSOAs

## Layering maps
Much like other mapping software it is possible to layer maps in Python. We will plot the bus stops on top of the leeds dataset.

We need to check the CRS of both datasets is the same, so we can accurately plot the layers on top of each other.

In [ ]:
# Check the CRS are the same
print(leeds.crs, bus_stops.crs)

## Plot the bus stops and LSOAs together

In [ ]:
# plot one subplot (1 map), with dimensions 16 X 8
f, ax = plt.subplots(1, figsize=(16, 8))

# define the basemap plot it on the sublot axis
base = leeds.plot(ax=ax)

# plot the bus stops on the basemap axis, colour the bus stops red
bus_stops.plot(ax=base,color='red')

# show the map
plt.show()

As we can see we are able to plot the bus stop data (for the whole of West Yorkshire) onto the Leeds Map.

## Choropleth Mapping
Choropleth maps are maps where the polygons are coloured different shades or colours based on a value. E.g. Number of people commuting by bus per LSOA

In [ ]:
# Define plot size
f,ax = plt.subplots(1, figsize=(16,8))
# Plot the LSOAs, specifying to plot the 'Bus, minibus or coach' column
# Add legend (legend =True)
leeds.plot(ax=ax, column ='Bus, minibus or coach', legend=True)
# show the map
plt.show()

<font color= 'orchid'> <b>Code your own choropleth map for 'Works mainly from home'<b></font><br>
    Answer at the end of the workbook

In [ ]:
# code your choropleth map here!

### Plotting Categorical variables
First let's generate a categorical variable. We might be interested in the most common type of public transport used in each LSOA.

Have a look at the <code>.idxmax()</code> documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html

- what are we doing here?
- what does <code>axis=1</code> do in the function?


Comment the code below to demonstrate you know what it is doing

In [ ]:
leeds['most_common_public_transit']=leeds[['Underground, metro, light rail, tram',
       'Train', 'Bus, minibus or coach', 'Taxi',
       'Motorcycle, scooter or moped', 'Bicycle', 'On foot']].idxmax(axis=1)

It is possible to create choropleth maps with categorical variables (e.g. most_common_public_transit). To do this we specify <code> categorical=True,</code> withn the <code>plot()</code> function.<br> 

Note the legend of the map is now separate colours and not a continuous colour bar

<font color ='orchid'> <b> Run the code below to plot a categorical choropleth </font>

 Let's improve this visualisation a bit.

In [ ]:
f,ax = plt.subplots(1, figsize=(16,8))
# plot the 'most_common_public_transit'column
## use the tab20_r palette, tabe20
leeds.plot(ax=ax, column ='most_common_public_transit', categorical=True, cmap='Set2', legend=True)
# get the axis for the legend
leg = ax.get_legend()
# move the legend axis to the bottom left corner. 
leg.set_bbox_to_anchor((0.3, 0.2))
# remove axis
ax.set_axis_off()
# set title
ax.set_title('Most common public transport used to commute (LSOA)')
# show plot
plt.show();

<code>leg.set_bbox_to_anchor((0.3, 0.2))</code> Try editing the coordinates in the <code>set_bbox_to_anchor()</code> function. See where the legend moves.

In [ ]:
leeds['WD24NM'].unique()

## Sub-setting spatial data
We can also work with and plot just a subset of the spatial data. For example we might only be interested in the Ward of 'Headingley & Hyde Park' in Leeds. In which case we can use the <code>.loc[]</code> function to locate all rows (LSOAs) where the ward is 'Headingley & Hyde Park'. Just like we would normally do in a non-spatial pandas data frame.<br>

<font color ='orchid'> <b>Run the code below </font>

In [ ]:
# locate LSOAs (rows) in the Ward ('WD24NM') of 'Headingley & Hyde Park'
hyde_park = leeds.loc[leeds['WD24NM']== 'Headingley & Hyde Park',:]
# view the newly created hyde park dataframe
hyde_park

It is now possible to plot just Hyde Park and Headingley. We can also put maps side by side to compare them. Here we are going to number of people commuting by bike versus on Food for each LSOA.<br>

<font color = 'orchid'> <b> Run the code below,</b> make sure you understand what each line does (there is a lot going on)</font>

In [ ]:
# create a figure with two subplots (maps)
f,ax = plt.subplots(1,2, figsize=(12,6))

# plot population estimate in subplot 1
hyde_park.plot(ax=ax[0], column ='On foot', legend=True)

# plot gdp estimate in subplot 2
hyde_park.plot(ax=ax[1], column ='Bicycle', legend=True)

# give subplot 1 an informative title
ax[0].set_title('On foot')

# give subplot 2 an informative title
ax[1].set_title('Bicycle')

# make axis invisible for subplot 1
ax[0].set_axis_off()

# make axis invisible for subplot 1
ax[1].set_axis_off()

# show figure
plt.show()

In [ ]:
## Lety's improve the plots further

# create a figure with two subplots (maps)
f,ax = plt.subplots(1,2, figsize=(12,6))


# Normalize data with a set center.
## https://matplotlib.org/3.2.0/api/_as_gen/matplotlib.colors.TwoSlopeNorm.html?highlight=twoslopenorm#matplotlib.colors.TwoSlopeNorm
# define the minimum and maximum limits of the cbar

# minimum = 0 (as count data)
vmin =0
# calculate the maximum value across both columns 
vmax = hyde_park[['On foot','Bicycle']].max().max()
# use the maximum bicycle value (the smaller of the two counts) as the midpoint of the colorbar
vcenter=hyde_park['Bicycle'].max()

# normalise the data with a center
from matplotlib import colors
divnorm=colors.TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)

# plot population estimate in subplot 1, 
# shrink legend and normalise cbar
hyde_park.plot(ax=ax[0], column ='On foot', legend=True, legend_kwds={'shrink': 0.4}, norm=divnorm)

# plot gdp estimate in subplot 2
# shrink legend and normalise cbar
hyde_park.plot(ax=ax[1], column ='Bicycle', legend=True, legend_kwds={'shrink': 0.4}, norm=divnorm)

# give subplot 1 an informative title
ax[0].set_title('Number of working population commuting on foot')

# give subplot 2 an informative title
ax[1].set_title('Number of working population commuting by bicycle')

# make axis invisible for subplot 1
ax[0].set_axis_off()

# make axis invisible for subplot 1
ax[1].set_axis_off()

# show figure
plt.show()

## Subplots and axes

A quick note on the subplot and ax indexing. It can be a little tricky getting your head around the indexing used when creating subplots.
To create multiple subplots you use the code below, specifying firstly the number of rows then the number of columns. <br>
<code>.subplots([number of rows], [number of columns])</code>

When you are then specifying each individual plot, you need to specify the axes of the plot, using the code <code> ax=ax[]</code>. The indexing for the axes starts at 0. I.e. the first row is 'row 0' and the first column, 'column 0'. This is illustrated below.

![](https://github.com/FrancescaPontin/GEOG5990M/blob/main/notebooks/screenshots/axes_diagram.png?raw=true)

For a set of subplots with just one row you only need to specify the column in the <code>ax=ax[]</code> function. <br>
I.e. <code>ax=ax[column number]</code> <br>

I.e.<code> plt.subplot(1,2) <br>
dataframe.plot(ax=ax[0], ...
dataframe.plot(ax=ax[1], ... </code>
![](https://github.com/FrancescaPontin/GEOG5990M/blob/main/notebooks/screenshots/axes_2.png?raw=true)

Similarly for a set of subplots with just one column you only need to specify the row in the <code>ax=ax[]</code> function. <br>
<code> plt.subplot(2,1) <br>
dataframe.plot(ax=ax[0], ...
dataframe.plot(ax=ax[1], ... </code>
</code>
![](https://github.com/FrancescaPontin/GEOG5990M/blob/main/notebooks/screenshots/axes_3.png?raw=true)

For a set of subplots with just multiple rows and columns you only need to specify both in the <code>ax=ax[,]</code> funciton.(row first then column)<br>
<code>plt.subplot(2,3)<br>
dataframe.plot(ax=ax[0,0], ...
dataframe.plot(ax=ax[0,1], ...
dataframe.plot(ax=ax[0,2], ...
dataframe.plot(ax=ax[1,0], ...
dataframe.plot(ax=ax[1,1], ...
dataframe.plot(ax=ax[1,2], ...</code>
![](https://github.com/FrancescaPontin/GEOG5990M/blob/main/notebooks/screenshots/axes_4.png?raw=true

<div class="alert alert-block alert-warning">
    
## Extra task: Using USA data

### The data

In [ ]:
usa_cities = gpd.read_file(gplt.datasets.get_path('usa_cities'))
contiguous_usa = gpd.read_file(gplt.datasets.get_path('contiguous_usa'))

# remove cities in sates not in the contiguous USA (not connected directly to the mainland), for ease of plotting
continental_usa_cities = usa_cities.loc[(usa_cities['STATE'] !="HI") & (usa_cities['STATE'] !="AK" ) & (usa_cities['STATE'] !="PR")]


In [ ]:
continental_usa_cities.head()

In [ ]:
continental_usa_cities.plot()

In [ ]:
contiguous_usa.head()

In [ ]:
contiguous_usa.plot()

### Tasks:
- check the CRS of both geo data frames
- produce a layered map of cities and states
- Produce a choropleth map of 2010 population
- produce a choropleth map of elevation
- Colour city by state
- subset the data to plot a single state e.g. Washington

# Exercise: Manipulating spatial data

## Geometric Manipulations

In [ ]:
#For the sake of this exercise we are going to change the projection of the data
bus_stops_p1 =bus_stops.to_crs('epsg:4326')


Let's compare CRS

In [ ]:
bus_stops.crs

Notice in the British National Grid projection(EPSG:27700) the axis information indicates the units used are metres: <br>

<code>Axis Info [cartesian]:</code><br>
<code> - E[east]: Easting (metre)</code><br>
<code> - N[north]: Northing (metre)</code>


In [ ]:
bus_stops_p1.crs


Notice in the WGS 84 (World Geodetic System) projection(EPSG:4326) the axis information indicates the units used are degrees: <br>

<code>Axis Info [ellipsoidal]:</code><br>
<code>- Lat[north]: Geodetic latitude (degree)</code><br>
<code>- Lon[east]: Geodetic longitude (degree)</code><br>


Projection therefore matters when we come to do spatial analysis with our data (just like in every other GIS)

### Buffers

A buffer in geographic information system (GIS) is a zone around a map feature measured in units of distance or time.

In Python we specify the buffer size: the radius of the buffer (in this case size is measured in meters as the projection is EPSG:27700).

E.g.:

<code> dataframe.buffer(distance =800) </code> distance = 800 meters.

800 meters is often used to represent a 5 minute walk.

In [ ]:
#Plot 800m meter arround bus stops

# plot one subplot (1 map), with dimensions 16 X 8
f, ax = plt.subplots(1, figsize=(16, 8))

bus_stops.buffer(distance=800).plot(ax=ax)

plt.show()

If the projection was ESG:4326 the buffer would be calculated using degrees- see what happens

In [ ]:
bus_stops_p1.buffer(distance=10).explore()

In [ ]:
# run the code without plotting- notice we generate an array of geometries 
bus_stops.buffer(distance=800)

In [ ]:
# we can check the data type- a geopandas array
bus_stops.buffer(distance=800).dtype

In order to keep information about the area being buffered in the newly created buffer geo-data-frame it is useful to copy the original data you want to conduct the geometric manipulation on, and name it something else e.g. dataframe_buffer.

Then replace the geometry column in the copied data with the calculated buffer geometry.

E.g.
<code> dataframe_buffer = dataframe.copy() </code>

<code> dataframe_buffer['geometry'] = dataframe.buffer(distance)</code>

In [ ]:
# copy the bus stops dataframe
bus_stops_buffer = bus_stops.copy()

# apply the function (replacing the geometry column with the buffer geometry)
bus_stops_buffer['geometry'] = bus_stops.buffer(distance=800)

bus_stops_buffer.head()

Note because we copied the bus_stops data frame above the bus_stops_buffer data frame still contains the name of the bus stop and other information e.g. average number of buses in the morning.


We can add the bus_stop_buffer layer to the map along with the bus_stop and leeds layers.

<font color = 'orchid'> <b> Run the code below, read the comments to understand what each line of code is doing  </font>

In [ ]:
# plot one subplot (1 map), with dimensions 16 X 8
f, ax = plt.subplots(1, figsize=(16, 8))

# define the basemap plot it on the sublot axis
base = leeds.plot(ax=ax, color='grey')

# plot the bus stop buffers on the basemap axis, colour buffers blue
bus_stops_buffer.plot(ax=base,color='blue', alpha=0.2)

# plot the bus stops on the basemap axis, colour the bus stops red
bus_stops.plot(ax=base,color='red', markersize=1)

# shw the map
plt.show()

### Centroids

Put simply the centroid is the center most point of a polygon (there are different debated methods of calculating centroids, but unless you are using centroids for a specific purpose the method should not matter too much).

#### Why calculate centroids?

Lots of geometric manipulations and analysis use centroids. For example you might use centroids to as a proxy to measure the distance between two polygons.


<font color = 'orchid'> <b>Use the code </b> <code> dataframe.centroid </code> <b>to find the LSOA centroids (run the next 3 cells of code for the worked example below) </font>

In [ ]:
# copy the leeds dataframe
leeds_centroid = leeds.copy()

# calcualte the centorid
# and replace the leeds geometry (polygon) with the centroid geometry (point)
leeds_centroid['geometry'] = leeds.centroid

# check - geometry should contain point data
leeds_centroid.head()

In [ ]:
# plot to check it looks as expected
leeds_centroid.plot();

In [ ]:
# plot one subplot (1 map), with dimensions 16 X 8
f, ax = plt.subplots(1, figsize=(16, 8))

# define the basemap plot it on the sublot axis
base = leeds.plot(ax=ax, color='grey')

# plot the lsoa centroids on the basemap axis, colour the centroids purple
leeds_centroid.plot(ax=base,color='purple')

# show the map
plt.show()

We can also change marker size and colour of plotted points to reflect the data they represent (as we did with the choropleth maps for polygon data).

<font color = 'orchid'> <b>Run the code below</font>

In [ ]:
# plot one subplot (1 map), with dimensions 16 X 8
f, ax = plt.subplots(1, figsize=(16, 8))

# define the basemap plot it on the sublot axis
base = leeds.plot(ax=ax, color='grey')

# plot the lsoa centroids on the basemap axis, colour and size the centroids based opn number of people commuting by bus
leeds_centroid.plot(ax=base, markersize='Bus, minibus or coach', column='Bus, minibus or coach',  legend=True)

# make axis invisible
ax.set_axis_off()

# show the map
plt.show()

We can improve this visualisation further by scaling the marker size, adjusting the legend extent and adding a title

<font color = 'orchid'> <b>Run and read the code below, check you understand what is happening on each line</font>

In [ ]:
# plot one subplot (1 map), with dimensions 16 X 8
f, ax = plt.subplots(1, figsize=(16, 8))

# define the basemap plot it on the sublot axis
base = leeds.plot(ax=ax, color='grey')

# plot the lsoa centroids on the basemap axis, colour and size the centroids based opn number of people commuting by bus
# scale the marker size to fit better and make make semi-opaque to see overlap in dense areas
# start colorbar legend at 0 as count data
leeds_centroid.plot(ax=base, markersize=leeds_centroid['Bus, minibus or coach']/4, column='Bus, minibus or coach', alpha=0.8, legend=True, vmin=0)

# make axis invisible
ax.set_axis_off()

# add title, increase fontsize
ax.set_title('Number of people commuting by bus in Leeds by LSOA', fontsize =14)

# show the map

### Convex hull polygons

A convex hull is the smallest polygon that you can draw around a collection of points/a polygon.

<font color = 'orchid'> <b>Run the code below to create convex hull polygons (using the code </b><code> dataframe.convex_hull</code><b> around the lsoas in Leeds </font>

In [ ]:
leeds_convex_hull = leeds.copy()

# replace the geometry column with the calculated convexhull polygon geometries 
leeds_convex_hull['geometry'] = leeds.convex_hull

# plot one subplot (1 map), with dimensions 16 X 8
f, ax = plt.subplots(1, figsize=(16, 8))

# define the basemap plot it on the sublot axis
base = leeds.plot(ax=ax, color='grey')

# plot the africa convex hull polygons
# .boundary alows us to just plot the polygon outline
leeds_convex_hull.boundary.plot(ax=base)

# make axis invisible
ax.set_axis_off()

# show the map
plt.show()

## Spatially aggregating data
Often our data will not be at the same spatial scale, so we may need to aggregate areas of data together to get them to the same spatial scale. Or we may only be interested in larger spatial trends. Therefore, we need to convert our smaller area data to larger area data. In geopandas we can easily so this using the <code>dissolve</code> function.
In this example we are going to aggregate LSOAs up to ward (WD24NM) level. <br>
Think of dissolve as removing all the internal LSOA boarders within the ward to leave just the ward outline.<br>
The data for the ward also gets aggregated, below we have aggregated all the numeric data <br>
<font color='orchid'> <b> Run the code below </font>

In [ ]:
# specify which columns from the leeds dataframe we are going to aggregate: 
# #this should include the column we are going to aggregate the data by (i.e. WD24NM), the geometry column and any other columns we want to aggregate
# use the WD24NM column to inform the dissolve, 
# in this case sum is a suitable aggregation to get the total within the wards, we could also use 'mean' to get the average within the wards
leeds_wards = leeds[['WD24NM','Total_distance_to_work', 'Less than 10km',
       '10km to less than 30km', '30km and over', 'Works mainly from home',
       'Not in employment or works mainly offshore, in no fixed place or outside the UK',
       'Work mainly at or from home', 'Underground, metro, light rail, tram',
       'Train', 'Bus, minibus or coach', 'Taxi',
       'Motorcycle, scooter or moped', 'Driving a car or van',
       'Passenger in a car or van', 'Bicycle', 'On foot',
       'Other method of travel to work',
       'Not in employment or aged 15 years and under','geometry']].dissolve(by='WD24NM', aggfunc='sum').reset_index()

# view the new leeds_wards dataframe
leeds_wards

In [ ]:
# plot the new leeds_wards df

leeds_wards.explore(column ='Bus, minibus or coach', vmin=0)

Think carefully about how you want to aggregate the columns, we can also assign an aggfunction to each column (as we did with <code>.agg()</code> when looking at the bike share data).

E.g.
<code>, aggfunc={'r_rank':'mean','r_exp':'max'})</code>

There is no quick way to assign multiple columns the same function so with a large dataset you might want to consider which columns you include in the aggregated spatial data frame.


#### Replicate and edit the code above to aggregate the LSOA data to plot mean number of people commuting by bike per ward 
(Answer at the end)

In [ ]:
# Write your code here

## Joining a non-spatial dataset to a spatial dataset

We are going to read in some data about the resident population of lsoas across the UK and join it to the Leeds data frame

<font color ='orchid'><b> Run the code below to read in a csv of lsoa level resident population data </font>

In [ ]:
# read in residential popualtion data from 2021 census
lsoa_resident_pop =pd.read_csv('https://github.com/FrancescaPontin/GEOG5990M/raw/refs/heads/main/data/week_6_7/lsoa_resident_pop.csv')
# use loc to subset data to only contain data for Leeds
leeds_resident_pop =lsoa_resident_pop.loc[lsoa_resident_pop['geography'].str.contains('Leeds'),:]
# have a quick look at the dataframe
leeds_resident_pop.head()

In [ ]:
print('Check both datasets have the same number of rows:', leeds_resident_pop.shape[0], leeds.shape[0])

In [ ]:
leeds.head()

The 'LSOA21CD' column in the Leeds data frame matches that of the 'geography code' column in the leeds_resident_pop data frame. As they are formatted in exactly the same way and each row is a unique LSOA, the geography code/ LSOA21CD is a unique identifier common to both data frames. Therefore, we will use this column to join our datasets. <br>
The code we will use is <code>pd.merge()</code>. We need to specify a few parameters within the function:<br>

- Firstly we specify which data frames we want to join (in the order we want to join them).<br>
<code> pd.merge(leeds, leeds_resident_pop... </code> <br>


- Secondly we need to specify the column in each data frame, leeds is on the left so we specify left_on='LSOA21CD': as we are using the 'LSOA21CD' column from the leeds dataframe. And the leeds_resident_pop data frame is on the right, so we specify right_on='geography code' as we are using the 'geography code' column for our join.<br>
<code> pd.merge(leeds, leeds_resident_pop, left_on='LSOA21CD', right_on='geography code' ... </code> <br>


- Finally, we need to specify how the tables are joined. This is based on [SQL join fomats](http://www.complexsql.com/sql-joins-2/). In this case we are using a left join (we keep all the data in the left data frame and just add the columns form the right dataframe on the end).
<code> pd.merge(leeds, leeds_resident_pop, left_on='LSOA21CD', right_on='geography code', how='left') </code> <br>

In [ ]:
leeds_pop = pd.merge(leeds, leeds_resident_pop, left_on='LSOA21CD', right_on='geography code', how='left')
leeds_pop.head()

In [ ]:
# check if any of the rows in the newly joined columns are NA, if so this might suggest that the join has not worked as expected 
leeds_pop['Residence type: Total; measures: Value'].isna().any()

Now we will visualise our newly joined data by plotting the total resident population in each lsoa

In [ ]:
f,ax = plt.subplots(1, figsize=(16,8))
leeds_pop.plot(column='Residence type: Total; measures: Value',legend=True,ax=ax)
# remove axis
ax.set_axis_off()
plt.show();

Because there is such a large variation in population in the above map with a continuous scale can be difficult to distinguish between differences in smaller population numbers. To make a more informative plot we can use <code> scheme = ' '</code>
And use <code> ['equal_interval', 'quantiles', 'fisher_jenks', 'fisher_jenks_sampled']</code> to define how the choropleth map is scaled.

In [ ]:
f,ax = plt.subplots(1, figsize=(16,8))
leeds_pop.plot(column='Residence type: Total; measures: Value',legend=True, cmap='summer',scheme='equal_interval', k=6, ax=ax)
# remove axis
ax.set_axis_off()
# position legend
ax.get_legend().set_bbox_to_anchor((.12, .4))
# add title
ax.set_title('Leeds total resident population (LSOA)', fontsize=15)
# add a North arrow (look at the documentation here: https://github.com/pmdscully/geo_northarrow)
add_north_arrow(ax=ax, scale=.75, xlim_pos=0.1, ylim_pos=.85, color='#000', text_scaler=2, text_yT=-1.25)
plt.show();

<font color='orchid'><b> Write your own code to map 'Residence type: Lives in a communal establishment; measures: Value'</font> <br>
Answer at the end

In [ ]:
# plot here!





## Spatially Joining Data

It is also possible to join data based on their spatial relationship to each other using <code>.sjoin()</code>
For example we might want to know the lsoa each bus stop is in.


Like with <code>pd.merge()</code> there are parameters we need to specify when using <code>gpd.sjoin()</code>

- Again firstly we specify which data frames we want to join (in the order we want to join them).<br>
<code>gpd.sjoin(bus_stops, leeds, ... </code> <br>


- Secondly we need to specify how the tables are joined. (Again based on [SQL join fomats](http://www.complexsql.com/sql-joins-2/)). In this case we are using an inner join
<code> gpd.sjoin(bus_stops, leeds, how="inner",... </code> <br>

- Finally, we need to specify the type of spatial join using 'predicate'. From the [Geopandas documentation](http://geopandas.org/mergingdata.html) <br>
<i>The `predicate' argument specifies how geopandas decides whether or not to join the attributes of one object to another. There are three different join options as follows:
    - <b>intersects:</b> The attributes will be joined if the boundary and interior of the object intersect in any way with the boundary and/or interior of the other object.
    - <b>within:</b> The attributes will be joined if the object’s boundary and interior intersect only with the interior of the other object (not its boundary or exterior).
    - <b>contains:</b> The attributes will be joined if the object’s interior contains the boundary and interior of the other object and their boundaries do not touch at all. </i> <br>
    
<font color = 'orchid'><b> Run the code below to spatially join the data </font>

In [ ]:
bus_stops_leeds = gpd.sjoin(bus_stops, leeds, how="inner", predicate='intersects')
# looka t the data: notice that we now have the LSOA level info for each bus stop 
bus_stops_leeds .head()

Note when mapped we now just have bus stops within Leeds, not the whole of West Yorkshire

In [ ]:
# let us plot the bus stops now coloured by the the number of people commuting by bus in that LSOA
f,ax = plt.subplots(1, figsize=(16,8))

# map the leeds LSOAs (basemap)
base = leeds.plot(ax=ax, color='grey')
# plot the bus stops and colour them based on n of people commuting by bus in the LSOA in which the bus stop is
bus_stops_leeds.plot(ax=base,  column ='Bus, minibus or coach', cmap='viridis',markersize=2, vmin=0,legend=True)
# remove axis
ax.set_axis_off()
plt.show()

## Overlay - creating spatial layers from the intersections, unions, and differences between map
I have included this for reference with a worked example, to show you how we can look at where spatial data overlaps. A list of the overlay operations can be found here: https://geopandas.org/set_operations.html
Have a read and explore the data.

In [ ]:
# get the intersection between the 5 minute walk of a bus stop buffer and lsoas to see how much of the LSOA is accessible by bus
lsoas_in_buff =gpd.overlay(bus_stops_buffer , leeds, how='intersection')
# look at the data
lsoas_in_buff 

In [ ]:
# plot ther results 
f,ax = plt.subplots(1, figsize=(16,8))

# lsoa base map
base = leeds.plot(ax=ax,color='grey')
# plot the intersection of bus stop buffer and lsoas 
lsoas_in_buff .plot(ax=base, alpha=0.8)
# remove axis
ax.set_axis_off()
plt.show();

### Kernel density map

Much like the kernel density plots we produced in day 1 this produces a kernel density estimate of spatial point data.

In [ ]:
## Seaborn example

f,ax = plt.subplots(1, figsize=(16,8))

# map the leeds (basemap)
base = leeds.plot(ax=ax, color='grey')

# map the kernel density estimate of bus stops
sns.kdeplot(bus_stops_leeds, x='x', y='y', color='Red', fill=True,ax=base, alpha=0.8)

# remove axis
ax.set_axis_off()

plt.show()

### Adding base maps
Base maps provide the reader with context for a map. At a smaller scale than we are currently using the basemap may show road networks or point of interest.

We are going to use contextily to add the background map to the geographic data using the <code>.basemap()</code> function.

#### Aligning the CRS
But first we need to convert the CRS of the data we want to plot to the Web Mercator projection (epsg=3857). So the base maps and geographic data we are plotting align.

Unless the data file I want to plot is particularly big I tend to save the data with the Web Mercator projection as a new geodataframe to avoid confusion.

In [ ]:
leeds.crs

In [ ]:
# convert CRS to epsg=3857 Web Mercator
leeds_WM = leeds.to_crs(epsg=3857)

fig,ax = plt.subplots(1, figsize=(10,10))

# plot the Leeds data as usual
leeds_WM.plot(column='Works mainly from home',ax=ax,alpha=0.5)

# add ctx basemap to ax
ctx.add_basemap(ax, crs=leeds_WM.crs)

ax.set_axis_off()

#### Different base maps.
By default ctx.add_basemap() uses the Stamen Terrain style.

To see the available different base maps we can get the provider keys:

List of contextily providers:

In [ ]:
ctx.providers.keys()

Each will have their own options, accessed by the code: <code> ctx.providers.<text color='red'>provider_from_above</font>.keys()</code>

E.g. <code> ctx.providers.CartoDB.keys()</code>

In [ ]:
ctx.providers.CartoDB.keys()

We can then use the provider and provider options to choose our base map e.g. <code>CartoDB.Voyager</code>

In [ ]:


f,ax = plt.subplots(1, figsize=(16,16))

# plot the Leeds data as usual
leeds.plot(figsize=(10, 10), alpha=0.5,column='Works mainly from home',ax=ax)

# add ctx basemap to ax, specifying the basemap provider and options
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Voyager, zoom=12, crs=leeds.crs)

ax.set_axis_off()


In [ ]:
f,ax = plt.subplots(1, figsize=(16,16))

# plot the Leeds data as usual
leeds_WM.plot(figsize=(10, 10), alpha=0.5,column='Works mainly from home',ax=ax)

# add ctx basemap to ax, specifying the basemap provider and options
ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter, zoom=12)

ax.set_axis_off()

<font color='orchid'><b>  Play around with the providers and options to get different base maps.

<h1> Exercise answers</h1>  <a class="tocSkip">

#### Plot Choropleth of number of people per LSOA working from home

In [ ]:
#### Plot Choropleth of number of people per LSOA working from home
# Define plot size
f,ax = plt.subplots(1, figsize=(16,8))
# Plot the LSOAs, specifying to plot the 'Works mainly from home' column
# Add legend (legend =True)
leeds.plot(ax=ax, column ='Works mainly from home', legend=True)
# show the map
plt.show()

#### Replicate and edit the code above to aggregate the LSOA data to plot mean number of people commuting by bike per ward 

In [ ]:
# specify which columns from the leeds data frame we are going to aggregate: 
# calcualte mean bike commuting per ward
leeds_wards_mean_bike = leeds[['WD24NM','Bus, minibus or coach', 'geometry']].dissolve(by='WD24NM', aggfunc='mean').reset_index()

# view the new leeds_wards dataframe
leeds_wards_mean_bike.explore('Bus, minibus or coach', vmin=0)

#### Write your own code to map 'Residence type: Lives in a communal establishment; measures: Value'

Your map might look different depending on the scheme and number of breaks you used!

In [ ]:
f,ax = plt.subplots(1, figsize=(16,8))
leeds_pop.plot(column='Residence type: Lives in a communal establishment; measures: Value',legend=True, cmap='summer',scheme='natural_breaks', k=5, ax=ax)
# remove axis
ax.set_axis_off()
# position legend
ax.get_legend().set_bbox_to_anchor((.12, .4))
# add title
ax.set_title('Leeds population living in a communal establishment (LSOA)', fontsize=15)
# add a North arrow (look at the documentation here: https://github.com/pmdscully/geo_northarrow)
add_north_arrow(ax=ax, scale=.75, xlim_pos=0.1, ylim_pos=.85, color='#000', text_scaler=2, text_yT=-1.25)
plt.show();
